In [1]:
import time
start = time.time()

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.utils.data import Subset

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Custom Loss Function (L1 loss + CrossEntropy)
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.l1 = nn.L1Loss()
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, predictions, targets):
        l1_loss = self.l1(predictions, torch.ones_like(predictions))  # Example L1 loss
        ce_loss = self.cross_entropy(predictions, targets)  # Cross entropy loss
        return l1_loss + ce_loss  # Combined loss

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Create the model, optimizer, and custom loss function
model = SimpleNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
custom_loss_fn = CustomLoss()

# Set up TensorBoard writer
writer = SummaryWriter('runs/custom_loss_experiment')

# Load the full dataset
dataset = datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor())

# Define the size of the subset (e.g., 10% of the full dataset)
subset_size = int(0.1 * len(dataset))  # 10% of the data
indices = torch.randperm(len(dataset))[:subset_size]  # Randomly choose a subset of indices

# Create a subset of the dataset
subset = Subset(dataset, indices)

# Create a new DataLoader for the subset
train_loader_subset = torch.utils.data.DataLoader(subset, batch_size=64, shuffle=True)

# Train using the subset
for epoch in range(2):  # Train for 2 epochs as an example
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader_subset):
        # Perform the training steps here
        #print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Data Shape: {data.shape}")
        
        data, target = data.to(device), target.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Calculate loss
        loss = custom_loss_fn(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Log loss to TensorBoard
        if batch_idx % 100 == 0:  # Log every 100 batches
            print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item()}')
            writer.add_scalar('Training Loss', loss.item(), epoch * len(train_loader_subset) + batch_idx)
            
    # Log average loss per epoch to TensorBoard
    writer.add_scalar('Epoch Average Loss', running_loss / len(train_loader_subset), epoch)

# Close the TensorBoard writer
writer.close()

Epoch 1, Batch 0, Loss: 3.2827978134155273
Epoch 2, Batch 0, Loss: 1.4558491706848145


In [2]:
%load_ext tensorboard

In [3]:
# Start TensorBoard within the Jupyter notebook
%tensorboard --logdir=runs


In [4]:
print(time.time() - start)

6.301794052124023
